# Script for a period

In [46]:
import requests
import json
import pandas as pd
from clickhouse_connect import get_client
from datetime import datetime, timedelta, date
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [47]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [48]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [49]:
def process_advert_data(data):
    # Flatten the JSON data
    df = pd.json_normalize(
        data['adverts'],
        record_path='advert_list', 
        meta=['type', 'status', 'count']
    )
    
    # Convert 'changeTime' to datetime format
    df['changeTime'] = pd.to_datetime(df['changeTime'])
    
    # Reset the index
    df = df.reset_index(drop=True)
    
    return df

# Now you can process each dataset with one line:
df_guten = process_advert_data(data_guten)
df_giper = process_advert_data(data_giper)
df_kitchen = process_advert_data(data_kitchen)
df_smart = process_advert_data(data_smart)

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [50]:
# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# Function to fetch campaign data
def fetch_campaign_data(df, headers, project_name, chunk_size=50):
    chunks = [df['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df), chunk_size)]
    all_campaign_data = []
    for idx, chunk in enumerate(chunks):
        response = requests.post(url, params=query_params, json=chunk, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_campaign_data.extend(data)
            print(f"Data retrieved successfully for {project_name} chunk {idx + 1}")
        else:
            print(f"Error for {project_name} chunk {idx + 1}: {response.status_code}, {response.text}")
        time.sleep(1)  # Add a delay to avoid hitting API rate limits
    
    all_campaign_data = pd.DataFrame(all_campaign_data)
    all_campaign_data = all_campaign_data.sort_values(by='createTime', ascending=False)
    all_campaign_data['Project'] = project_name
    all_campaign_data['Marketplace'] = 'Wildberries'
    
    return all_campaign_data

# Fetch campaign data for each project
campaigns_guten = fetch_campaign_data(df_guten, headers_guten, 'WB-GutenTech')
campaigns_giper = fetch_campaign_data(df_giper, headers_giper, 'WB-ГиперМаркет')
campaigns_kitchen = fetch_campaign_data(df_kitchen, headers_kitchen, 'WB-KitchenAid')
campaigns_smart = fetch_campaign_data(df_smart, headers_smart, 'WB-Smart-Market')

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

print("Combined Campaign Data")
#campaigns_kitchen

Data retrieved successfully for WB-GutenTech chunk 1
Data retrieved successfully for WB-GutenTech chunk 2
Data retrieved successfully for WB-GutenTech chunk 3
Data retrieved successfully for WB-GutenTech chunk 4
Data retrieved successfully for WB-GutenTech chunk 5
Data retrieved successfully for WB-GutenTech chunk 6
Data retrieved successfully for WB-GutenTech chunk 7
Data retrieved successfully for WB-GutenTech chunk 8
Data retrieved successfully for WB-GutenTech chunk 9
Data retrieved successfully for WB-ГиперМаркет chunk 1
Data retrieved successfully for WB-KitchenAid chunk 1
Data retrieved successfully for WB-Smart-Market chunk 1
Combined Campaign Data


In [51]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-05-14,2025-05-14,CASO_Су-вид_СЛИВ_14.05.2025_3%,25507095,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
1,2100-01-01,2025-05-14,2025-05-14,CASO_Тостеры_СЛИВ_14.05.2025_3%,25506832,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2025-05-14,2025-05-14,CASO_Электрооткрывалка_СЛИВ_14.05.2025_3%,25506544,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
3,2100-01-01,2025-05-14,2025-05-14,CASO_Капучинатор_СЛИВ_14.05.2025_3%,25504578,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
4,2100-01-01,2025-05-14,2025-05-14,CASO_Пакеты для упаковщиков_СЛИВ_14.05.2025_3%,25504089,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
494,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
495,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
496,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
497,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [52]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"
# Automatically get yesterday's date
yesterday = "2025-05-12"
specific_date = str(yesterday)  # Replace with your desired date

### Guten

In [53]:
def fetch_campaign_stats(campaign_ids, headers, date, base_url,project_name, chunk_size=100):
    chunks = [campaign_ids['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaign_ids), chunk_size)]
    all_data = []
    for idx, chunk in enumerate(chunks): 
        #Construct the payload for the current chunk
        payload = [{"id": campaign_id, "dates": [date]} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(base_url, headers=headers, json=payload)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1} for {project_name}" )
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
            if "no companies with correct intervals" in response.text:
                print("Stopping execution due to invalid interval error")
                break
            
        # Add a delay to avoid hitting API rate limits
        time.sleep(65)
            
    return all_data if all_data else [{}] 

# Fetch campaign data
all_campaign_data_guten = fetch_campaign_stats(campaigns_guten, headers_guten, specific_date, url, "WB-GutenTech")
all_campaign_data_giper = fetch_campaign_stats(campaigns_giper, headers_giper, specific_date, url, "WB-ГиперМаркет")
all_campaign_data_kitchen = fetch_campaign_stats(campaigns_kitchen, headers_kitchen, specific_date, url, "WB-KitchenAid")
all_campaign_data_smart = fetch_campaign_stats(campaigns_smart, headers_smart, specific_date, url, "WB-Smart-Market")


Data retrieved successfully for chunk 1 for WB-GutenTech
Data retrieved successfully for chunk 2 for WB-GutenTech
Error for chunk 3: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Stopping execution due to invalid interval error
Data retrieved successfully for chunk 1 for WB-ГиперМаркет
Data retrieved successfully for chunk 1 for WB-KitchenAid
Data retrieved successfully for chunk 1 for WB-Smart-Market


In [54]:
all_campaign_data_giper

[{'views': 8217,
  'clicks': 382,
  'ctr': 4.65,
  'cpc': 3.49,
  'sum': 1334.81,
  'atbs': 91,
  'orders': 26,
  'cr': 6.81,
  'shks': 26,
  'sum_price': 82340,
  'dates': ['2025-05-12'],
  'days': [{'date': '2025-05-12T03:00:00+03:00',
    'views': 8217,
    'clicks': 382,
    'ctr': 0,
    'cpc': 3.49,
    'sum': 1334.81,
    'atbs': 91,
    'orders': 26,
    'cr': 6.81,
    'shks': 26,
    'sum_price': 82340,
    'apps': [{'views': 1197,
      'clicks': 40,
      'ctr': 3.34,
      'cpc': 4.67,
      'sum': 186.96,
      'atbs': 8,
      'orders': 1,
      'cr': 2.5,
      'shks': 1,
      'sum_price': 3016,
      'nm': [{'views': 78,
        'clicks': 3,
        'ctr': 3.85,
        'cpc': 4.77,
        'sum': 14.31,
        'atbs': 2,
        'orders': 0,
        'cr': 0,
        'shks': 0,
        'sum_price': 0,
        'name': 'Тонометр автоматический UA-777 большая манжета 32-45 см',
        'nmId': 235999757},
       {'views': 743,
        'clicks': 20,
        'ctr': 2.69,


In [55]:
all_campaign_data_smart

[{'views': 2906,
  'clicks': 131,
  'ctr': 4.51,
  'cpc': 2.61,
  'sum': 341.6,
  'atbs': 16,
  'orders': 5,
  'cr': 3.82,
  'shks': 5,
  'sum_price': 85512,
  'dates': ['2025-05-12'],
  'days': [{'date': '2025-05-12T03:00:00+03:00',
    'views': 2906,
    'clicks': 131,
    'ctr': 0,
    'cpc': 2.61,
    'sum': 341.6,
    'atbs': 16,
    'orders': 5,
    'cr': 3.82,
    'shks': 5,
    'sum_price': 85512,
    'apps': [{'views': 455,
      'clicks': 11,
      'ctr': 2.42,
      'cpc': 5.24,
      'sum': 57.61,
      'atbs': 4,
      'orders': 2,
      'cr': 18.18,
      'shks': 2,
      'sum_price': 36125,
      'nm': [{'views': 84,
        'clicks': 2,
        'ctr': 2.38,
        'cpc': 5.24,
        'sum': 10.48,
        'atbs': 2,
        'orders': 1,
        'cr': 50,
        'shks': 1,
        'sum_price': 29977,
        'name': 'Электробритва Series 9 Pro+ 9510s',
        'nmId': 235599972},
       {'views': 113,
        'clicks': 2,
        'ctr': 1.77,
        'cpc': 7.45,
    

In [56]:
def flatten_campaign_data(campaign_data):
    flattened_data = []
    
    for entry in campaign_data:
        advertId = entry.get("advertId")
        for day in entry.get("days", []):
            date = day.get("date")
            for app in day.get("apps", []):
                for nm in app.get("nm", []):
                    flattened_data.append({
                        "date": date,
                        "nmId": nm.get("nmId"),
                        "name": nm.get("name"),
                        "views": nm.get("views"),
                        "clicks": nm.get("clicks"),
                        "ctr": nm.get("ctr"),
                        "cpc": nm.get("cpc"),
                        "sum": nm.get("sum"),
                        "atbs": nm.get("atbs"),
                        "orders": nm.get("orders"),
                        "cr": nm.get("cr"),
                        "shks": nm.get("shks"),
                        "sum_price": nm.get("sum_price"),
                        "advertId": advertId,
                    })
    
    # Create DataFrame
    df = pd.DataFrame(flattened_data)
    
    # Convert and clean date column
    if not df.empty and "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None)
    
    return df

# Process all datasets with one line each
df_guten = flatten_campaign_data(all_campaign_data_guten)
df_giper = flatten_campaign_data(all_campaign_data_giper)
df_kitchen = flatten_campaign_data(all_campaign_data_kitchen)
df_smart = flatten_campaign_data(all_campaign_data_smart)

df_guten


,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-05-12 03:00:00,110511380,Эпилятор женский электрический Silk-epil 3-176,31,0,0.00,0.00,4.38,0,0,0.00,0,0,21204098
1,2025-05-12 03:00:00,236556054,Эпилятор женский электрический Silk-epil 3-031,45,1,2.22,6.62,6.62,0,0,0.00,0,0,21204098
2,2025-05-12 03:00:00,236558115,Эпилятор женский электрический Silk-epil 3-000,48,1,2.08,6.72,6.72,0,0,0.00,0,0,21204098
3,2025-05-12 03:00:00,236559228,Эпилятор женский электрический Silk-epil 1-010,39,0,0.00,0.00,5.66,0,0,0.00,0,0,21204098
4,2025-05-12 03:00:00,236558116,Эпилятор женский электрический Silk-epil 3-202,38,1,2.63,5.50,5.50,0,0,0.00,0,0,21204098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,2025-05-12 03:00:00,252275487,Вытяжка кухонная 60 см встраиваемая телескопич...,485,12,2.47,4.12,49.43,1,0,0.00,0,0,19297685
658,2025-05-12 03:00:00,252275488,Вытяжка кухонная 60 см встраиваемая телескопич...,60,1,1.67,6.63,6.63,0,0,0.00,0,0,19297685
659,2025-05-12 03:00:00,252275489,Вытяжка кухонная 60 см встраиваемая телескопич...,368,11,2.99,3.43,37.77,0,1,9.09,1,5985,19297685
660,2025-05-12 03:00:00,252275487,Вытяжка кухонная 60 см встраиваемая телескопич...,82,6,7.32,1.40,8.37,0,0,0.00,0,0,19297685


In [57]:
def group_and_aggregate(df, project_name):

    grouped_df = (
        df.groupby([df["date"].dt.date, "nmId","advertId"], as_index=False)
        .agg({
            "date": "first",  # Keep the first date (to retain the day)
            "name": "first",  # Keep the first name (or customize this logic)
            "views": "sum",
            "clicks": "sum",
            "ctr": "mean",  # Sum or average, depending on your needs
            "cpc": "mean",  # Use mean for 'cpc' (cost per click)
            "sum": "sum",
            "atbs": "sum",
            "orders": "sum",
            "cr": "mean",  # Use mean for 'cr' (conversion rate)
            "shks": "sum",
            "sum_price": "sum",
            "advertId": "first"  # Keep the first 'advertId' (no summing)
        })
    )
    
    # Add project and marketplace info
    grouped_df['Project'] = project_name
    grouped_df['Marketplace'] = 'Wildberries'
    
    return grouped_df

# Process each DataFrame using the function
df_grouped_guten = group_and_aggregate(df_guten, 'WB-GutenTech')
df_grouped_giper = group_and_aggregate(df_giper, 'WB-ГиперМаркет')
df_grouped_kitchen = group_and_aggregate(df_kitchen, 'WB-KitchenAid')
df_grouped_smart = group_and_aggregate(df_smart, 'WB-Smart-Market')

# Combine all DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)


# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,date,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,25088481,2025-05-12 03:00:00,Эпилятор женский электрический Silk-epil 9-880,433,18,3.686667,6.343333,96.22,4,0,0.000000,0,0,21204285,WB-GutenTech,Wildberries
1,25623718,2025-05-12 03:00:00,Бритва мужская электрическая S3 300BT с триммером,277,8,2.883333,2.836667,27.97,0,0,0.000000,0,0,21202453,WB-GutenTech,Wildberries
2,25623718,2025-05-12 03:00:00,Бритва мужская электрическая S3 300BT с триммером,439,28,5.456667,1.793333,89.01,4,2,4.786667,2,14342,21202745,WB-GutenTech,Wildberries
3,25624340,2025-05-12 03:00:00,Электробритва Series 3 300s,639,25,2.886667,1.420000,61.49,1,0,0.000000,0,0,21202453,WB-GutenTech,Wildberries
4,25624340,2025-05-12 03:00:00,Электробритва Series 3 300s,719,70,9.156667,3.113333,145.15,8,1,1.190000,1,6064,21202745,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,235597032,2025-05-12 03:00:00,Эпилятор женский электрический Silk-epil 7-030,705,28,3.906667,3.600000,94.58,0,0,0.000000,0,0,20822563,WB-Smart-Market,Wildberries
462,235599972,2025-05-12 03:00:00,Электробритва Series 9 Pro+ 9510s,694,27,3.473333,3.656667,80.30,2,1,16.666667,1,29977,15562363,WB-Smart-Market,Wildberries
463,235604574,2025-05-12 03:00:00,Триммер для бороды и усов BT5421,211,5,1.833333,3.230000,28.12,2,0,0.000000,0,0,19597465,WB-Smart-Market,Wildberries
464,235638522,2025-05-12 03:00:00,Эпилятор Silk-epil 5 5-000,8,0,0.000000,0.000000,0.55,0,0,0.000000,0,0,20822563,WB-Smart-Market,Wildberries


In [58]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final

,nmId,date,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25088481,2025-05-12 03:00:00,Эпилятор женский электрический Silk-epil 9-880,433,18,96.22,4,0,0,0,21204285,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион
1,25623718,2025-05-12 03:00:00,Бритва мужская электрическая S3 300BT с триммером,277,8,27.97,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
2,25623718,2025-05-12 03:00:00,Бритва мужская электрическая S3 300BT с триммером,439,28,89.01,4,2,2,14342,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
3,25624340,2025-05-12 03:00:00,Электробритва Series 3 300s,639,25,61.49,1,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
4,25624340,2025-05-12 03:00:00,Электробритва Series 3 300s,719,70,145.15,8,1,1,6064,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,235597032,2025-05-12 03:00:00,Эпилятор женский электрический Silk-epil 7-030,705,28,94.58,0,0,0,0,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания
462,235599972,2025-05-12 03:00:00,Электробритва Series 9 Pro+ 9510s,694,27,80.30,2,1,1,29977,15562363,WB-Smart-Market,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания
463,235604574,2025-05-12 03:00:00,Триммер для бороды и усов BT5421,211,5,28.12,2,0,0,0,19597465,WB-Smart-Market,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания
464,235638522,2025-05-12 03:00:00,Эпилятор Silk-epil 5 5-000,8,0,0.55,0,0,0,0,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания


## Статистика карточек товаров по дням
https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail/history

Response Schema: application/json

- data / Array of objects 
    - nmID / integer <int32> / Артикул W
    - imtName / string / Наименование карточки товар
    - vendorCode/ string/ Артикул продавц
    - history/ Array of objects / Array
        - dt/ string <date> / Дата
        - openCardCount / integer <int32> / Количество переходов в карточку товар
        - addToCartCount / integer <int32> / Положили в корзину, шту
        - ordersCount / integer <int32> / Заказали товаров, ш
        - ordersSumRub / integer <int32> / Заказали на сумму, руб/ buyoutsCount/ integer <int32/ Выкупили товаров, шт
        - buyoutsSumRub / integer <int32> / Выкупили на сумму, руб
        - buyoutPercent / integer <int32> / Процент выкупа, % (Какой процент посетителей, заказавших товар, его выкупили. Без учёта товаров, которые еще доставляются покупателю.)
        - addToCartConversion / numbe / Конверсия в корзину, % (Какой процент посетителей, открывших карточку товара, добавили товар в корзину)
        - cartToOrderConversion / integer <int32> / Конверсия в заказ, % (Какой процент посетителей, добавивших товар в корзину, сделали заказ)
- error/ boolean / Флаг ошибки
- errorText/ string / Описание ошибки
- additionalErrors / Array of objects / Дополнительные ошибки
        - field/ string / Структура, где допущена ошибки
        - description/ string / Описание

In [59]:
# API endpoint
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail/history"

# Define headers and project names
projects = {
    'WB-GutenTech': headers_guten,
    'WB-ГиперМаркет': headers_giper,
    'WB-KitchenAid': headers_kitchen,
    'WB-Smart-Market': headers_smart
}

# Calculate yesterday's date
#yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

# Define the period for the report (only yesterday)
period = {
    "begin": yesterday,
    "end": yesterday
}

# Function to send a single POST request
def send_request(nm_ids, period, headers):
    request_body = {
        "nmIDs": nm_ids,
        "period": period
    }
    response = requests.post(url, headers=headers, json=request_body)
    return response

# Function to fetch data in batches
def fetch_data_in_batches(nm_ids, period, headers, project_name):
    all_data = []
    batch_size = 20  # Maximum allowed nmIDs per request
    requests_per_minute = 3  # Maximum allowed requests per minute
    interval = 60 / requests_per_minute  # Time interval between requests in seconds

    for i in range(0, len(nm_ids), batch_size):
        batch = nm_ids[i:i + batch_size]
        response = send_request(batch, period, headers)

        if response.status_code == 200:
            try:
                data = response.json()
                if not data.get('error') and 'data' in data:
                    all_data.extend(data['data'])
                    print(f"Data retrieved successfully for batch {i // batch_size + 1} for {project_name}")
                else:
                    print(f"Error in response for batch {i // batch_size + 1}: {data.get('errorText', 'No error text')}")
            except ValueError as e:
                print(f"Failed to decode JSON for batch {i // batch_size + 1}: {e}")
        else:
            print(f"Error for batch {i // batch_size + 1}: {response.status_code}, {response.text}")
            break

        time.sleep(interval)  # Respect the rate limit

    return all_data

# List to store DataFrames for each project
dataframes = []

# Process each project
for project_name, headers in projects.items():
    # Filter the DataFrame for the current project
    filtered_df = df_final[df_final['Project'] == project_name]
    unique_nmId_values = filtered_df['nmId'].unique().tolist()
    print(f"Total unique nmId values for {project_name}:", len(unique_nmId_values))  # Debugging line

    # Fetch data for all unique nmId values
    all_data = fetch_data_in_batches(unique_nmId_values, period, headers, project_name)

    # Flatten the nested 'history' data for easier analysis
    flattened_data = []
    for item in all_data:
        nmID = item['nmID']
        imtName = item['imtName']
        vendorCode = item['vendorCode']
        for history in item['history']:
            history_entry = {
                'nmID': nmID,
                'imtName': imtName,
                'vendorCode': vendorCode,
                'dt': history['dt'],
                'openCardCount': history['openCardCount'],
                'addToCartCount': history['addToCartCount'],
                'addToCartConversion': history['addToCartConversion'],
                'ordersCount': history['ordersCount'],
                'ordersSumRub': history['ordersSumRub'],
                'cartToOrderConversion': history['cartToOrderConversion'],
                'buyoutsCount': history['buyoutsCount'],
                'buyoutsSumRub': history['buyoutsSumRub'],
                'buyoutPercent': history['buyoutPercent']
            }
            flattened_data.append(history_entry)

    # Convert the flattened data to a DataFrame
    df_copy = pd.DataFrame(flattened_data)
    print(f"Data for {project_name}:\n", df_copy.head())

    # Add the DataFrame to the list
    dataframes.append(df_copy)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
print("Combined DataFrame:\n", combined_df.head())
df_copy=combined_df.copy()

Total unique nmId values for WB-GutenTech: 187
Data retrieved successfully for batch 1 for WB-GutenTech
Data retrieved successfully for batch 2 for WB-GutenTech
Data retrieved successfully for batch 3 for WB-GutenTech
Data retrieved successfully for batch 4 for WB-GutenTech
Data retrieved successfully for batch 5 for WB-GutenTech
Data retrieved successfully for batch 6 for WB-GutenTech
Data retrieved successfully for batch 7 for WB-GutenTech
Data retrieved successfully for batch 8 for WB-GutenTech
Data retrieved successfully for batch 9 for WB-GutenTech
Data retrieved successfully for batch 10 for WB-GutenTech
Data for WB-GutenTech:
        nmID                                            imtName vendorCode  \
0  25088481     Эпилятор женский электрический Silk-epil 9-880   Б0035166   
1  25623718  Бритва мужская электрическая S3 300BT с триммером   Б0046737   
2  25624340                        Электробритва Series 3 300s   Б0027735   
3  25628178               Бритва мужская электриче

In [60]:
df_copy=combined_df.copy()
df_final_copy=df_final.copy()

In [61]:
# Convert 'day' and 'dt' to datetime for accurate merging
df_final_copy['date'] = pd.to_datetime(df_final_copy['date']).dt.date
df_copy['dt'] = pd.to_datetime(df_copy['dt']).dt.date

# Rename columns in df2 to match df1 for merging
df_copy.rename(columns={'nmID': 'nmId', 'dt': 'day'}, inplace=True)
df_final_copy.rename(columns={'date': 'day'}, inplace=True)

# Merge the DataFrames on 'nmId', 'day', and 'Project'
merged_df_2 = pd.merge(
    df_final_copy,
    df_copy[['nmId', 'day', 'ordersCount', 'ordersSumRub','addToCartCount']],
    on=['nmId', 'day'],
    how='left'
)

# Fill NaN values with 0 for ordersCount and ordersSumRub
merged_df_2['ordersCount'].fillna(0, inplace=True)
merged_df_2['ordersSumRub'].fillna(0, inplace=True)
merged_df_2['addToCartCount'].fillna(0, inplace=True)


# Display the merged DataFrame
merged_df_2

C:\Users\User\AppData\Local\Temp\ipykernel_9276\3378984998.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df_2['ordersCount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_9276\3378984998.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,...,Marketplace,endTime,createTime,startTime,name_campaign,status,type,ordersCount,ordersSumRub,addToCartCount
0,25088481,2025-05-12,Эпилятор женский электрический Silk-epil 9-880,433,18,96.22,4,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион,0,0,6
1,25623718,2025-05-12,Бритва мужская электрическая S3 300BT с триммером,277,8,27.97,0,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,3,21513,4
2,25623718,2025-05-12,Бритва мужская электрическая S3 300BT с триммером,439,28,89.01,4,2,2,14342,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,3,21513,4
3,25624340,2025-05-12,Электробритва Series 3 300s,639,25,61.49,1,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,2,12128,12
4,25624340,2025-05-12,Электробритва Series 3 300s,719,70,145.15,8,1,1,6064,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,2,12128,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,235597032,2025-05-12,Эпилятор женский электрический Silk-epil 7-030,705,28,94.58,0,0,0,0,...,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,1,10638,2
462,235599972,2025-05-12,Электробритва Series 9 Pro+ 9510s,694,27,80.30,2,1,1,29977,...,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания,1,29977,3
463,235604574,2025-05-12,Триммер для бороды и усов BT5421,211,5,28.12,2,0,0,0,...,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания,0,0,2
464,235638522,2025-05-12,Эпилятор Silk-epil 5 5-000,8,0,0.55,0,0,0,0,...,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,0,0,0


In [62]:
merged_df_2

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,...,Marketplace,endTime,createTime,startTime,name_campaign,status,type,ordersCount,ordersSumRub,addToCartCount
0,25088481,2025-05-12,Эпилятор женский электрический Silk-epil 9-880,433,18,96.22,4,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион,0,0,6
1,25623718,2025-05-12,Бритва мужская электрическая S3 300BT с триммером,277,8,27.97,0,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,3,21513,4
2,25623718,2025-05-12,Бритва мужская электрическая S3 300BT с триммером,439,28,89.01,4,2,2,14342,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,3,21513,4
3,25624340,2025-05-12,Электробритва Series 3 300s,639,25,61.49,1,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,2,12128,12
4,25624340,2025-05-12,Электробритва Series 3 300s,719,70,145.15,8,1,1,6064,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,2,12128,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,235597032,2025-05-12,Эпилятор женский электрический Silk-epil 7-030,705,28,94.58,0,0,0,0,...,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,1,10638,2
462,235599972,2025-05-12,Электробритва Series 9 Pro+ 9510s,694,27,80.30,2,1,1,29977,...,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания,1,29977,3
463,235604574,2025-05-12,Триммер для бороды и усов BT5421,211,5,28.12,2,0,0,0,...,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания,0,0,2
464,235638522,2025-05-12,Эпилятор Silk-epil 5 5-000,8,0,0.55,0,0,0,0,...,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,0,0,0


## Insert the data

In [63]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [65]:
# Ensure date columns are in the correct format for ClickHouse
merged_df_2['day'] = pd.to_datetime(merged_df_2['day'])  # Convert to datetime


# Debugging: Check the data types of the DataFrame
print("Data types of merged_df:")
print(merged_df_2.dtypes)

# Ensure the DataFrame has the correct columns
columns = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks',
    'sum_price', 'advertId', 'Project', 'Marketplace','endTime', 'createTime', 'startTime', 
    'name_campaign', 'status', 'type','ordersCount', 'ordersSumRub','addToCartCount'
]

# Reorder columns to match the expected order
merget_df_copy_2 = merged_df_2[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in merget_df_copy_2.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'campaign_data_wb'

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

Data types of merged_df:
nmId                       int64
day               datetime64[ns]
name_product              object
views                      int64
clicks                     int64
sum                      float64
atbs                       int64
orders                     int64
shks                       int64
sum_price                  int64
advertId                   int64
Project                   object
Marketplace               object
endTime                   object
createTime                object
startTime                 object
name_campaign             object
status                    object
type                      object
ordersCount                int64
ordersSumRub               int64
addToCartCount             int64
dtype: object
Sample data to insert: [(25088481, Timestamp('2025-05-12 00:00:00'), 'Эпилятор женский электрический Silk-epil 9-880', 433, 18, 96.22, 4, 0, 0, 0, 21204285, 'WB-GutenTech', 'Wildberries', datetime.date(2100, 1, 1), datetime.date(2024, 